In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from generateGif import create_animated_gif
from geom import find_point_C
import random
import math
from simulation import SimulationParams, sim
from scipy.optimize import minimize
from targetFunction import targetFunction

In [2]:
params = SimulationParams()
params.AB_length = 1.1
params.DC_length = 2.2
params.BC_length = 3.2
params.DA_x = 0
params.DA_y = 3
params.BCE_inner_angle = 0.5  # радиан
params.CE_length = 1
params.alpha_angle = 0.2  # радиан

X, Y = sim(params)
create_animated_gif(
    X,
    Y,
    [0, 1, 2, 2, 4, 5],
    [1, 2, 3, 4, 5, 6],
    "tests.gif",
    3000,
    text="Проверка генерации гифки на основе результатов симуляции",
)

Анимированный GIF сохранен как tests.gif.


In [3]:
def objective_function(x):
    # Создаём параметры симуляции
    params = SimulationParams()
    params.AB_length = x[0]
    params.DC_length = x[1]
    params.BC_length = x[2]
    params.DA_x = x[3]
    params.DA_y = x[4]
    params.BCE_inner_angle = x[5]  # радиан
    params.CE_length = x[6]
    params.alpha_angle = x[7]  # радиан

    try:
        # Проводим симуляцию
        X, Y = sim(params)

        # Вычисляем целевую функцию
        obj = targetFunction(Y[:, 5])
    except:
        obj = np.inf
    return obj


def optimize_function(bounds):
    for i in range(5):
        # Начальная точка для оптимизации
        initial_guess = np.zeros(len(bounds))
        for j in range(len(bounds)):
            initial_guess[j] = random.uniform(*bounds[j])

        # Оптимизация
        result = minimize(objective_function, initial_guess, bounds=bounds)

        # Проверка на успех
        if result.success:
            params = SimulationParams()
            params.AB_length = result.x[0]
            params.DC_length = result.x[1]
            params.BC_length = result.x[2]
            params.DA_x = result.x[3]
            params.DA_y = result.x[4]
            params.BCE_inner_angle = result.x[5]  # радиан
            params.CE_length = result.x[6]
            params.alpha_angle = result.x[7]  # радиан
            X, Y = sim(params)
            print("Optimization success")
            create_animated_gif(
                X,
                Y,
                [0, 1, 2, 2, 4],
                [1, 2, 3, 4, 5],
                f"candidate_{i}.gif",
                3000,
                text=f"T={result.fun} |AB|={params.AB_length} |DC| |BC|\n\
                    |CE|={params.CE_length} DA_x={params.DA_x} \
                    DA_y={params.DA_y} BCE={params.BCE_inner_angle}rad alpha={params.alpha_angle}rad",
            )
        else:
            print("Optimization failed")


bounds = [
    (0, 50),
    (0, 300),
    (0, 300),
    (-300, 300),
    (0, 300),
    (-math.pi / 2, math.pi / 2),
    (-math.pi / 2, 300),
    (0, math.pi / 2),
]  # Пределы для x1 и x2
optimize_function(bounds)

Optimization success
Анимированный GIF сохранен как candidate_0.gif.
Optimization failed


C:\Users\zhugeo\AppData\Roaming\Python\Python312\site-packages\scipy\optimize\_numdiff.py:592: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Optimization success
Анимированный GIF сохранен как candidate_2.gif.
Optimization success
Анимированный GIF сохранен как candidate_3.gif.
Optimization failed
